konlpy 설치
- 환경변수 편집 jvm.dll 경로 확인 : C:\Program Files\Java\jdk1.8.0_261\jre\bin\server

https://data-scientist-brian-kim.tistory.com/79


In [1]:
%pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import konlpy

In [5]:
import pandas as pd

train_df = pd.read_csv('./dataset/ratings_train.txt',sep='\t')
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [6]:
test_df = pd.read_csv('./dataset/ratings_test.txt',sep='\t')
test_df

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [13]:
train_df.loc[4].document

'사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다'

In [14]:
train_df['label'].value_counts( )

0    75173
1    74827
Name: label, dtype: int64

In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [19]:
# 데이터 전처리
import re

train_df = train_df.fillna(' ')
# 숫자를 공백으로
train_df['document'] = train_df['document'].apply(lambda x : re.sub(r"\d+"," ",x))
# test 데이터 전처리
test_df = pd.read_csv('./dataset/ratings_test.txt',sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x: re.sub(r"\d+"," ",x))
train_df.drop('id', axis=1, inplace=True) 
test_df.drop('id', axis=1, inplace=True)

In [35]:
from konlpy.tag import Okt
okt = Okt()
def Okt_tokenizer(text):
    # 입력 인자로 들어온 text를 형태소 단어로 토큰화하여 list 객체 반환
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

tfidf_vect = TfidfVectorizer(tokenizer=Okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

C:\Users\admin\anaconda3\envs\cakd5\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [39]:
import warnings
warnings.filterwarnings('ignore')

In [40]:
# [과제] lr으로 모델링 및 하이퍼 파라미터 튜닝 후 테스트 데이터로 평가하세요.
lr_clf = LogisticRegression(random_state=123)

# GridSearchCV를 사용해 하이터 파라미터 튜닝
params = {
    'C' : [1,3.5,4.5,5.5,10]
    }
grid_clf = GridSearchCV(lr_clf,param_grid=params,cv=3,scoring='accuracy',verbose=1)
grid_clf.fit(tfidf_matrix_train,train_df['label'])
print('최적 파라미터 : ',grid_clf.best_params_,'\n',
      '최적 파라미터 정확도: ',round(grid_clf.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
최적 파라미터 :  {'C': 3.5} 
 최적 파라미터 정확도:  0.8592


In [41]:
# 테스트 데이터로 평가
from sklearn.metrics import accuracy_score

# 훈련데이터 벡터변환한 객체를 활용해 테스트 데이터 벡터변환
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# GridSearchCV의 최적 파라미터로 예측 수행
pred = grid_clf.best_estimator_.predict(tfidf_matrix_test)

print('테스트 데이터 정확도 : ',accuracy_score(test_df['label'],pred))

테스트 데이터 정확도 :  0.86188
